In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score,ConfusionMatrixDisplay
from sklearn.decomposition import PCA 
import seaborn as sb
from sklearn.preprocessing import StandardScaler,LabelEncoder
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import  Dense,LSTM,SimpleRNN
import os
import keras_tuner
import warnings
warnings.filterwarnings("ignore")



In [ ]:
df = pd.read_csv("emotions.csv")

x = df.loc[:,"fft_0_b":"label"]
y = df['label']
x.drop(columns='label',inplace=True,axis=1)
n_optimal = 45
model = PCA(n_components=45)
transformed_data =PCA.fit_transform(model,X=x)
scale = StandardScaler() 
x = scale.fit_transform(transformed_data)

encode = LabelEncoder()
y_ = encode.fit_transform(y)
x_train,x_test,y_train,y_test = train_test_split(x,y_,train_size=0.8,random_state=42)


In [92]:
activation  = ['relu','tanh']
loss_functions = [
    'binary_crossentropy',
    'sparse_categorical_crossentropy',
    'categorical_crossentropy',
    'kullback_leibler_divergence',
    'hinge',
    'weighted_categorical_crossentropy',
    'focal_loss',
    'squared_hinge'
]

Optimizer = [
            'adam',
            'sgd',
            'rmsprop',
            'adadelta']
def unit_set(hp):
    model = Sequential()
    
    model.add(SimpleRNN(units= 45,
                        activation=hp.Choice('activation_1',activation),
                        return_sequences=True,
                        input_shape=(45, 1)))
    
    model.add(SimpleRNN(units=30,
                        activation=hp.Choice('activation_2,',activation)))
    
    model.add(Dense(3,
                    activation='softmax')) 
    
    model.compile(optimizer =hp.Choice('optimize',Optimizer),
                  loss = hp.Choice('loss',Loss),
                  metrics=['accuracy'])
    return model

tune = keras_tuner.GridSearch(hypermodel=unit_set,
                              objective=['val_accuracy'],
                              overwrite=True,
                              directory='RRN_HyperParameter_Data_With_PCA',
                              project_name = 'LossAndOptimizer')
tune.search(x_train,y_train,epochs=15,validation_data = (x_test,y_test))


Trial 48 Complete [00h 00m 01s]

Best multi_objective So Far: -0.9344262480735779
Total elapsed time: 00h 09m 19s
INFO:tensorflow:Oracle triggered exit


In [94]:
hyp_param = tune.get_best_hyperparameters(num_trials=1)[0].values
print(hyp_param.keys())

dict_keys(['activation_1', 'activation_2,', 'optimize', 'loss'])


In [96]:
def unit_set(hp):
    model = Sequential()
    unit1 = hp.Int('unit1',min_value = 45, max_value = 65, step = 10)
    unit2 = hp.Int('unit2',min_value = 10, max_value = 50, step = 10)
    model.add(SimpleRNN(units= unit1,
                        activation=hyp_param['activation_1'],
                        return_sequences=True,
                        input_shape=(45, 1)))
    
    model.add(SimpleRNN(units=unit2,
                        activation=hyp_param['activation_2,']))
    
    model.add(Dense(3,
                    activation='softmax')) 
    
    model.compile(optimizer =hyp_param['optimize'],
                  loss = hyp_param['loss'],
                  metrics=['accuracy'])
    return model

tune = keras_tuner.GridSearch(hypermodel=unit_set,
                              objective=['val_accuracy'],
                              overwrite=True,
                              directory='RRN_HyperParameter_Data_With_PCA',
                              project_name = 'Units')

if not os.path.exists("RRN_HyperParameter_Data_With_PCA/Uints"):
    tune.search(x_train,y_train,epochs=15,validation_data = (x_test,y_test))


Trial 15 Complete [00h 00m 40s]
multi_objective: -0.9391100406646729

Best multi_objective So Far: -0.9391100406646729
Total elapsed time: 00h 10m 10s
INFO:tensorflow:Oracle triggered exit


In [109]:
model = tune.get_best_models(num_models=1)[0]
history = model.fit(x_train,y_train,epochs=15,batch_size=40)
model.save("RNN_with_pca.h5")


Epoch 1/15
43/43 [==============================] - 2s 13ms/step - loss: 0.0696 - accuracy: 0.9812
Epoch 2/15
43/43 [==============================] - 1s 13ms/step - loss: 0.0491 - accuracy: 0.9859
Epoch 3/15
43/43 [==============================] - 1s 12ms/step - loss: 0.0528 - accuracy: 0.9830
Epoch 4/15
43/43 [==============================] - 1s 12ms/step - loss: 0.0493 - accuracy: 0.9853
Epoch 5/15
43/43 [==============================] - 1s 12ms/step - loss: 0.0537 - accuracy: 0.9842
Epoch 6/15
43/43 [==============================] - 1s 13ms/step - loss: 0.0390 - accuracy: 0.9894
Epoch 7/15
43/43 [==============================] - 1s 13ms/step - loss: 0.0283 - accuracy: 0.9924
Epoch 8/15
43/43 [==============================] - 1s 12ms/step - loss: 0.0294 - accuracy: 0.9941
Epoch 9/15
43/43 [==============================] - 0s 12ms/step - loss: 0.0178 - accuracy: 0.9959
Epoch 10/15
43/43 [==============================] - 0s 11ms/step - loss: 0.0104 - accuracy: 0.9988
Epoch 11/